<a href="https://colab.research.google.com/github/andryll/Cortisol-Estimator-with-DL/blob/main/notebooks/CortiRocket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
  pip install aeon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 48.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


## Carregando o repositório do GitHub com os dados

In [3]:
!git clone https://github.com/andryll/Cortisol-Estimator-with-DL

Cloning into 'Cortisol-Estimator-with-DL'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 29 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 2.14 MiB | 3.96 MiB/s, done.
Resolving deltas: 100% (5/5), done.


## Carregando e Preparando os Dados


1.   Os dados de treino, teste e validação são carregados e concatenados, de modo a desfazer a divisão feita anteriormente.

2.   Em seguida, os dados de teste são normalizados com o MinMaxScaler



In [42]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

data_X = np.load("/content/Cortisol-Estimator-with-DL/data/X_train.npy")
data_X = np.concatenate((data_X, np.load("/content/Cortisol-Estimator-with-DL/data/X_val.npy")), axis=0)
data_X = np.concatenate((data_X, np.load("/content/Cortisol-Estimator-with-DL/data/X_test.npy")), axis=0)
print("data_X shape:", data_X.shape)

data_y = np.load("/content/Cortisol-Estimator-with-DL/data/y_train.npy")
data_y = np.concatenate((data_y, np.load("/content/Cortisol-Estimator-with-DL/data/y_val.npy")), axis=0)
data_y = np.concatenate((data_y, np.load("/content/Cortisol-Estimator-with-DL/data/y_test.npy")), axis=0)

scaler = MinMaxScaler()
data_y = scaler.fit_transform(data_y.reshape(-1, 1)).flatten()

print("data_y shape:", data_y.shape)

data_X shape: (148, 587, 4)
data_y shape: (148,)


## Realizando a Regressão com o ROCKET

In [45]:
from aeon.regression.convolution_based import RocketRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

# Realizando o KFold split com 10 folds
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42) # shuffle=True for random splits, random_state for reproducibility

mses = [] # Lista para armazenar o MSE de cada Fold

print(f"Realizando o {n_splits}-fold cross-validation...")

# Loop iterando cada uma das pastas
for fold, (train_index, test_index) in enumerate(kf.split(data_X)): # Enumerando as pastas
    print(f"\nFold {fold+1}/{n_splits}")

    # Dividindo os dados de acordo com a pasta atual
    X_train_fold, X_test_fold = data_X[train_index], data_X[test_index]
    y_train_fold, y_test_fold = data_y[train_index], data_y[test_index]

    # Inicializando o RocketRegressor com 10000 kernels
    reg = RocketRegressor(n_kernels=10000)
    # Treinando o Regressor
    reg.fit(X_train_fold, y_train_fold)

    # Realizando as predições
    y_pred_fold = reg.predict(X_test_fold)

    # Calculando o MSE para a pasta atual
    mse_fold = mean_squared_error(y_test_fold, y_pred_fold)
    mses.append(mse_fold)
    print(f"MSE para a pasta {fold+1}: {mse_fold}")

# Calculando a média dos MSE
average_mse = np.mean(mses)
print(f"\nAverage MSE across {n_splits} folds: {average_mse}")

Realizando o 10-fold cross-validation...

Fold 1/10
MSE para a pasta 1: 0.022975670330999808

Fold 2/10
MSE para a pasta 2: 0.06422727852680657

Fold 3/10
MSE para a pasta 3: 0.07333763661032557

Fold 4/10
MSE para a pasta 4: 0.06751828185184279

Fold 5/10
MSE para a pasta 5: 0.03433182813552613

Fold 6/10
MSE para a pasta 6: 0.014719649318364336

Fold 7/10
MSE para a pasta 7: 0.041361855876712324

Fold 8/10
MSE para a pasta 8: 0.03620251586981525

Fold 9/10
MSE para a pasta 9: 0.030185401048121712

Fold 10/10
MSE para a pasta 10: 0.08903839513319067

Average MSE across 10 folds: 0.04738985127017052
